In [11]:
!pip install pyngrok==7.1.6 fastapi uvicorn nest_asyncio streamlit > /dev/null
!pip install aiofiles pandas > /dev/null


In [12]:
%%writefile backend_app.py
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import time

app = FastAPI(title="Graph-RAG Backend")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def root():
    return {"message": "Graph-RAG backend running"}

@app.post("/qa")
async def qa(request: Request):
    data = await request.json()
    q = data.get("question", "")
    time.sleep(1)
    return {
        "answer": f"Mock answer for query: {q}",
        "latency": 1.02,
        "hops": 2,
        "citations": ["paper_1.pdf", "dataset_A.csv"]
    }


Writing backend_app.py


In [13]:
%%writefile app.py
import streamlit as st
import requests, time

st.set_page_config(page_title="Graph-RAG — Track B", layout="wide")
st.title("🧠 Graph-RAG — Track B (Frontend)")

backend_url = st.text_input("Enter backend URL", value="", placeholder="Paste backend URL here")
query = st.text_input("Enter your query:", "Which author proposed Method Y?")
if st.button("Run"):
    if backend_url.strip():
        try:
            start = time.time()
            res = requests.post(f"{backend_url}/qa", json={"question": query})
            latency = round(time.time() - start, 3)
            if res.status_code == 200:
                ans = res.json()
                st.success(ans["answer"])
                st.write(f"⏱️ Latency: {latency}s | 🔁 Hops: {ans['hops']} | 📚 Citations: {', '.join(ans['citations'])}")
            else:
                st.error(f"Request failed: HTTP {res.status_code}")
        except Exception as e:
            st.error(f"Error: {e}")
    else:
        st.warning("Please enter a backend URL first.")


Overwriting app.py


In [14]:
from pyngrok import ngrok
import threading, os, time, nest_asyncio
nest_asyncio.apply()

# Kill old ports
!fuser -k 8000/tcp || true
!fuser -k 8501/tcp || true

# Create TWO distinct tunnels
backend_tunnel = ngrok.connect(8000, bind_tls=True)
frontend_tunnel = ngrok.connect(8501, bind_tls=True)

print(f"✅ Backend running at: {backend_tunnel.public_url}")
print(f"✅ Frontend running at: {frontend_tunnel.public_url}\n")

def run_backend():
    os.system("uvicorn backend_app:app --host 0.0.0.0 --port 8000")

def run_frontend():
    os.system("streamlit run app.py --server.port 8501 --server.address 0.0.0.0")

threading.Thread(target=run_backend).start()
time.sleep(3)
threading.Thread(target=run_frontend).start()

print("🌐 App is starting... You can now open the frontend URL above.")


8501/tcp:             6114
✅ Backend running at: https://precongestive-elsy-pendente.ngrok-free.dev
✅ Frontend running at: https://precongestive-elsy-pendente.ngrok-free.dev

🌐 App is starting... You can now open the frontend URL above.


In [16]:
import requests, pandas as pd, time

BACKEND_URL = "https://precongestive-elsy-pendente.ngrok-free.dev"

queries = [
    "What was the impact of Hurricane Irma on Florida agriculture?",
    "Explain how AR/VR can enhance human–robot collaboration.",
    "Describe bias detection methods used in the DICES framework.",
    "What preprocessing is done in the PharmaLLM pipeline?",
    "How does the Quantum CNN differ from the Quantum SVM?"
]

results = []
for q in queries:
    start = time.time()
    try:
        res = requests.post(f"{BACKEND_URL}/qa", json={"question": q}, timeout=60)
        latency = round(time.time() - start, 3)
        data = res.json() if res.status_code == 200 else {}
        results.append({
            "question": q,
            "answer_preview": data.get("answer", "ERROR")[:100],
            "latency_s": latency,
            "hops": data.get("hops"),
            "num_citations": len(data.get("citations", []))
        })
    except Exception as e:
        results.append({
            "question": q, "answer_preview": f"Error: {e}",
            "latency_s": None, "hops": None, "num_citations": None
        })

df = pd.DataFrame(results)
df.to_csv("ablation_results_week7.csv", index=False)
print("✅ Evaluation complete! Results saved to ablation_results_week7.csv")
display(df)


✅ Evaluation complete! Results saved to ablation_results_week7.csv


,question,answer_preview,latency_s,hops,num_citations
0,What was the impact of Hurricane Irma on Flori...,Mock answer for query: What was the impact of ...,1.108,2.0,2
1,Explain how AR/VR can enhance human–robot coll...,ERROR,0.093,NaN,0
2,Describe bias detection methods used in the DI...,ERROR,0.099,NaN,0
3,What preprocessing is done in the PharmaLLM pi...,Mock answer for query: What preprocessing is d...,1.094,2.0,2
4,How does the Quantum CNN differ from the Quant...,Mock answer for query: How does the Quantum CN...,1.098,2.0,2
